In [2]:
import numpy as np
import scipy
import daqp
import numpy as np
from ctypes import * 
import ctypes.util
from sympy import Matrix



In [3]:
# final methods with comments

def is_invertible(A):
    return A.shape[0] == A.shape[1] and np.linalg.matrix_rank(A) == A.shape[0]

def fix_component(lam,lam_star,mu, W, B, p):
    print("run: fix component")
    print("B in fixed component",B)
    print("-lam[B]/p[B]",-lam[B]/p[B])
    j = np.argmin(-lam[B]/p[B])
    print("j",j)
    W_new = [int(i) for i in W if int(i)!=int(B[j])]
    print("W",W)
    #W_new = np.delete(W, B[j])
    print("new W", W_new)
    print("lam",lam)
    print("W[j]",W[j])
    print("(lam[B[j]]/p[B[j]])",(lam[B[j]]/p[B[j]]))
    lam = lam - (lam[B[j]]/p[B[j]])*p
    print("new lam",lam)
    lam_star[B[j]] = 0
    print("new lam_star")
    #mu[B[j]] = 0
    return lam,lam_star,mu, W_new          
            
    print("Done")

In [42]:
H = np.array([[1, 0], [0, 1]],dtype=c_double)
f = np.array([2, 2],dtype=c_double)
A = np.array([[1, 0], [0, 1]],dtype=c_double)
bupper = np.array([1,1],dtype=c_double)
blower= np.array([-1,-1],dtype=c_double)
sense = np.array([0,0],dtype=c_int)

A_new = np.array([[1, 0], [0, 1],[-1, 0], [0, -1]],dtype=c_double)
b_new = np.array([1,1,1,1],dtype=c_double)

In [118]:
# Thesis
H = np.array([[1, 1], [1, 4]],dtype=c_double)
f = np.array([1, -14],dtype=c_double)
A = np.array([[-2, 1], [-1, 3]],dtype=c_double)
bupper = np.array([7,11],dtype=c_double)
blower= np.array([0,0],dtype=c_double)
sense = np.array([0,0],dtype=c_int)
b = bupper
print(b.shape,f.shape,sense.shape)
#A_new = np.array([[1, 0], [0, 1],[-1, 0], [0, -1]],dtype=c_double)
#b_new = np.array([1,1,1,1],dtype=c_double)

(2,) (2,) (2,)


In [41]:
# Test repo
H = np.array([[1, 0], [0, 1]], dtype=c_double)
f = np.array([1, 1], dtype=c_double)
A = np.array([[1, 1], [1, -1]], dtype=c_double)
A = np.array([[1, 0], [0, 1],[1, 1], [1, -1]], dtype=c_double)
bupper = np.array([1, 2, 3, 4], dtype=c_double)
blower = np.array([-1, -2, -3, -4], dtype=c_double)
sense = np.array([0, 0, 0, 0], dtype=c_int)
xstar, fval, exitflag, lam = daqp.solve(H, f, A, bupper, blower, sense)


In [12]:
# try a different dimension
H = np.array([[1, 1], [2, 3]],dtype=c_double)
f = np.array([1, -14],dtype=c_double)
A = np.array([[-2, 1], [-1, 3],[1,-3]],dtype=c_double)
bupper = np.array([7,11,4],dtype=c_double)
blower= np.array([-100,-100,-100],dtype=c_double)
sense = np.array([0,0,0],dtype=c_int)

print(H.dtype,f.dtype,A.dtype,bupper.dtype)
print(H.shape,f.shape,A.shape,bupper.shape,sense.shape)

float64 float64 float64 float64
(2, 2) (2,) (3, 2) (3,) (3,)


In [4]:
n = 2 # Number of decision variables
m = 5 # Number of constraints
nth = 2 # Number of parameters

# Objective function
M = np.random.randn(n,n)
H = M @ M.T # Ensure H symmetric and positive definite. 
f = np.random.randn(n)
F = np.random.randn(n,nth)

# Constraints
A = np.random.randn(m,n)
b = np.random.rand(2*m)
T = np.random.randn(n,nth)#  A transformation such that x = F0*th is primal feasible
B = np.vstack((A,-A)) @ (-T)

In [5]:
""" Example QP of the form 

minimize_x  0.5 x' H x + ftot'x
subject to  A x <= btot

for a given theta
"""
theta = np.random.randn(nth)
btot = b + B @ theta
ftot = f + F @ theta

print(theta)
print(btot.shape)

[-0.29610038  0.00894474]
(10,)


In [8]:
import daqp
sense = np.zeros(btot.shape,c_int)
blower = np.array([-np.inf for i in range(10)],dtype=c_double)
x,fval,exitflag,info = daqp.solve(H,ftot,A,btot,blower,sense)
print("Optimal solution:")
print(x)
print("Exit flag:",exitflag)
print("Info:",info)

float64 float64 float64 float64 int32
(2, 2) (2,) (5, 2) (10,) (10,)
Optimal solution:
[-0.25362881 -0.36603138]
Exit flag: 1
Info: {'solve_time': 2.9e-06, 'setup_time': 5.2e-06, 'iterations': 5, 'nodes': 1, 'lam': array([0.        , 0.78056364, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.07827224, 0.        , 0.        ])}


: 

: 

In [130]:


## with print comments to debug

import sys

def daqp_self(H,f,A,b,sense,W):
    # transform matrices into M,d,v,R
    try:
        R = np.linalg.cholesky(H, upper = True)
    except:
        print("Invalid input")
        sys.exit()
    M = A @ np.linalg.inv(R)
    v = np.linalg.inv(R).T @f
    print("initial values")
    print("R",R,R.shape)
    print("M", M, M.shape)
    print("v", v,v.shape)
    print("b",b,b.shape)
    print("v", v,v.shape)
    d = b.reshape(-1) + M @ v
    print("d",d,d.shape)
    print("---------")
    # initial values
    lam = np.array([0 for i in range(len(b))],dtype=c_double)
    mu = np.array([0 for i in range(len(b))],dtype=c_double)
    lam_star =  np.array([0 for i in range(len(b))],dtype=c_double)
    
    print("initial lambda", lam)
    print("initial working set", W)
    print("mu",mu)
    iter = 0
    while True:
        if iter == 5:
            break
        iter += 1
        print("---------")
        print("Iteration",iter)
        W = np.sort(W)
        W = list(W)
        print("W",W, "len(W)", len(W))
        W_bar =  np.sort([x for x in range(len(lam)) if x not in W])
        print("W_bar", W_bar)
        print("mu at the beginning of the round",mu)
        p = np.array([0 for i in range(len(b))],dtype=c_double)
        if len(W)==0 or is_invertible(M[W]@ M[W].T): # step 2
            print("case: MkMkT invertible")
            print("lam_star",lam_star)
            print("lam",lam)
            print("len(W)",len(W))
            if len(W)>0:
                print("np.linalg.inv(M[W] @M[W].T)",np.linalg.inv(M[W,:] @M[W,:].T), (np.linalg.inv(M[W,:] @M[W,:].T)).shape)
                print("- np.linalg.inv(M[W] @M[W].T) @d[W]",- np.linalg.inv(M[W,:] @M[W,:].T) @d[W], (- np.linalg.inv(M[W,:] @M[W,:].T) @d[W]).shape)
                lam_star[W] = (- np.linalg.inv(M[W,:] @M[W,:].T) @d[W]) #.reshape(-1) # solve the system, step 3
            else:
                lam_star=np.array([0 for i in range(len(b))],dtype=c_double)
            print("lam",lam)
            print("lam_star",lam_star)
            if (np.array(lam_star) >= -1e-12).all(): # step 4
                print("case: all lam_star>=0")
                print("mu before update", mu)
                if len(W)>0:
                    if len(W_bar)>0: # constraints are split between W and W_bar
                        print("lam_star[W]",lam_star[W], lam_star[W].shape)
                        print("d[W_bar]",d[W_bar],d[W_bar].shape)
                        print(" M[W_bar] @M[W].T", M[W_bar,:] @M[W,:].T, (M[W_bar,:] @M[W,:].T).shape)
                        print("M[W_bar] @M[W].T@lam_star[W]",M[W_bar,:] @M[W,:].T@lam_star[W],(M[W_bar,:] @M[W,:].T@lam_star[W]).shape)
                        print("M[W_bar] @M[W].T@lam_star[W]+d[W_bar]",M[W_bar,:] @M[W,:].T@lam_star[W]+d[W_bar],(M[W_bar,:] @M[W,:].T@lam_star[W]+d[W_bar]).shape )
                        mu[W_bar] = M[W_bar,:] @M[W,:].T@lam_star[W]+d[W_bar]
                    else: # all constraints in W
                        print("case: all constraints in W")
                        print("CHECK THIS CASE")
                        mu = np.array([0 for i in range(len(b))],dtype=c_double)
                elif len(W)==0: # all constraints in W_bar
                    print("case: all constraints in W_bar")    
                    mu = np.copy(d)
                print("mu after update",mu)
                print("update lam now")
                print("mu",mu)
                lam = np.copy(lam_star) # optimum found ?
                print("lam",lam)
                if (np.array(mu) >= -1e-6).all(): # step 6
                    # optimum found
                    print("case: all mu >=0")
                    print("print all optimal values",R,M,lam_star, W,iter)
                    print("M[W,:]",M[W,:].shape)
                    print("lam_star[W]",lam_star[W].shape)
                    print("v",v.shape)
                    print("(M[W,:].T@lam_star[W]+v)",(M[W,:].T@lam_star[W]+v),(M[W,:].T@lam_star[W]+v).shape)
                    # problem: algorithm doesn't stop, when working set contains everything
                    return -np.linalg.inv(R)@(M[W,:].T@lam_star[W]+v), lam_star, W, iter
                else: # step 7
                    print("case: not all mu >=0")
                    print("mu_W_bar",mu[W_bar])
                    j = np.argmin(mu[W_bar])
                    print("j",W_bar[j])
                    W.append(int(W_bar[j]))
                    mu[j]=0
                    print("new W for next round",W)
            else:
                print("case: lam_star<0") # step 8
                print("lam_star",lam_star)
                print("lam",lam)
                p = lam_star-lam # step 9
                print("p",p)
                B = [i for i in W if lam_star[i]<1e-12]
                print("B",B)
                lam,lam_star,mu, W = fix_component(lam,lam_star,mu, W, B, p) # step 10
                print("lm_star",lam_star)
        else:
            print("W",W)
            print("M[W]",M[W,:])
            MM=Matrix(M[W,:] @ M[W,:].T) # step 12
            print("MM",MM)
            print("nullspace",MM.nullspace())
            nullspace = np.asarray(MM.nullspace()).reshape(-1)
            print("length of MM nullspace",len(nullspace))
            if len(nullspace)==0:
                p[W] = np.zeros((1,len(W)))
            else:
                for i in nullspace: #  (iterate over potential vectors in the nullspace)
                    print("element of nullspace",i)
                    pp = p
                    pp[W] = i
                    print("pp",pp.T)
                    print("d",d)
                    print("multiplication",pp.T@d)
                    if pp.T@d< 1e-12: 
                        print("case: test constraint") 
                        p[W] = i
                        break

            print("p",p)
            
            B = [i for i in W if p[i]<0] #step 13
            lam,lam_star,mu, W =  fix_component(lam,lam_star,mu,W,B,p) # step 14
            print("lam_star",lam_star)


H = np.array([[1, 1,0.5], [1, 4,1],[0.5,1,0.5]],dtype=c_double)
f = np.array([1, -12,1],dtype=c_double)
A = np.array([[0,-2, 1], [1,-1, 3],[-2,1,1.5],[-0.5,0,3]],dtype=c_double)
bupper = np.array([7,11,8,11],dtype=c_double)
blower= np.array([0,0],dtype=c_double)
sense = np.array([0,0],dtype=c_int)
b = bupper
        
x_star, lam_star, W, iter = daqp_self (H,f,A,b,eq,[1])
print("final outcome", x_star, lam_star,W, "iterations", iter)


opt_x,fval,exitflag,info = daqp.solve(H,f,A,bupper,blower,sense)
print("DAQP", opt_x,exitflag)
print(info)

 
  


initial values
R [[1.         1.         0.5       ]
 [0.         1.73205081 0.28867513]
 [0.         0.         0.40824829]] (3, 3)
M [[ 0.         -1.15470054  3.26598632]
 [ 1.         -1.15470054  6.94022094]
 [-2.          1.73205081  4.89897949]
 [-0.5         0.28867513  7.75671752]] (4, 3)
v [ 1.         -7.5055535   6.53197265] (3,)
b [ 7. 11.  8. 11.] (4,)
v [ 1.         -7.5055535   6.53197265] (3,)
d [37. 66. 25. 59.] (4,)
---------
initial lambda [0. 0. 0. 0.]
initial working set [1]
mu [0. 0. 0. 0.]
---------
Iteration 1
W [np.int64(1)] len(W) 1
W_bar [0 2 3]
mu at the beginning of the round [0. 0. 0. 0.]
case: MkMkT invertible
lam_star [0. 0. 0. 0.]
lam [0. 0. 0. 0.]
len(W) 1
np.linalg.inv(M[W] @M[W].T) [[0.01980198]] (1, 1)
- np.linalg.inv(M[W] @M[W].T) @d[W] [-1.30693069] (1,)
lam [0. 0. 0. 0.]
lam_star [ 0.         -1.30693069  0.          0.        ]
case: lam_star<0
lam_star [ 0.         -1.30693069  0.          0.        ]
lam [0. 0. 0. 0.]
p [ 0.         -1.306930

In [83]:
opt_x,fval,exitflag,info = daqp.solve(H,f,A,bupper,blower,sense)
print("DAQP", opt_x,exitflag)
print(info)

DAQP [-1.74226804  3.01030928] 1
{'solve_time': 2.1e-06, 'setup_time': 3e-06, 'iterations': 2, 'nodes': 1, 'lam': array([0.        , 0.        , 1.13402062])}


In [33]:
# final algorithm

import sys

def daqp_self(H,f,A,b,sense,W):
    # transform matrices into M,d,v,R
    try:
        R = np.linalg.cholesky(H, upper = True)
    except:
        print("Invalid input")
        sys.exit()
    M = A @ np.linalg.inv(R)
    v = np.linalg.inv(R).T @f
    d = b + M @ v
    # initial values
    lam = np.array([1 for i in range(len(b))],dtype=c_double)
    mu = np.array([-np.inf for i in range(len(b))],dtype=c_double)
    lam_star =  np.array([0 for i in range(len(b))],dtype=c_double)
    iter = 0
    while True:
        iter += 1
        W = np.sort(W)
        W = list(W)
        W_bar =  np.sort([x for x in range(len(lam)) if x not in W])
        p = np.array([0,0],dtype=c_double)
        if len(W)==0 or is_invertible(M[W]@ M[W].T): # step 2
            if len(W)>0:
                lam_star[W] = - np.linalg.inv(M[W] @M[W].T) @d[W] # solve the system, step 3
            else:
                lam_star=np.array([0,0],dtype=c_double)
            if (np.array(lam_star) >= 0).all(): # step 4
                if len(W)>0:
                    if len(W_bar)>0: # constraints are split between W and W_bar
                        mu[W] = M[W_bar] @M[W].T@lam_star[W]+d[W_bar]
                    else: # all constraints in W_bar
                        mu = np.array([0,0],dtype=c_double)
                elif len(W)==0: # all constraints in W
                    mu = np.copy(d)
                lam = np.copy(lam_star) # optimum found ?
                if (np.array(mu) >= 0).all(): # step 6
                    # optimum found
                    # problem: algorithm doesn't stop, when working set contains everything
                    return -np.linalg.inv(R)@(M[W].T@lam_star[W]+v), lam_star, W, iter
                else: # step 7
                    print("mu",mu)
                    print(W_bar)
                    j = np.argmin(mu[W_bar])
                    W.append(int(W_bar[j]))
            else: # step 8
                p = lam_star-lam # step 9
                B = [i for i in W if lam_star[i]<0]
                lam,lam_star,mu, W = fix_component(lam,lam_star,mu, W, B, p) # step 10
        else:
            MM=Matrix(M[W] @ M[W].T) # step 12
            nullspace = np.asarray(MM.nullspace()).reshape(-1)
            if len(nullspace)==0:
                p[W] = np.zeros((1,len(W)))
            else:
                for i in nullspace: #  (iterate over potential vectors in the nullspace)
                    pp = p
                    pp[W] = i
                    if pp.T@d< 0: 
                        p[W] = i
                        break
            
            B = [i for i in W if p[i]<0] #step 13
            lam,lam_star,mu, W =  fix_component(lam,lam_star,mu,W,B,p) # step 14


W = [] 
x_star, lam_star, W, iter = daqp_self(H,f,A,bupper,sense,W)  
print(f"x*={x_star}, lam*={lam_star},W={W}, #iterations: {iter}")
        

[-inf -inf -inf]
mu [-17.5 -20.   35. ]
[0 1 2]
mu [-17.5          0.27777778  35.        ]
[0]
run: fix component
B in fixed component [np.int64(0)]
-lam[B]/p[B] [0.]
0
new W [1]
lam [0.         2.22222222]
new lam [0.         2.22222222]
new lam_star
x*=[-1.83333333  3.05555556], lam*=[0.         2.22222222],W=[np.int64(1)], #iterations: 4


In [30]:
## with print comments to debug

import sys

def daqp_self(H,f,A,b,sense,W):
    # transform matrices into M,d,v,R
    try:
        R = np.linalg.cholesky(H, upper = True)
    except:
        print("Invalid input")
        sys.exit()
    M = A @ np.linalg.inv(R)
    v = np.linalg.inv(R).T @f
    d = b + M @ v
    print("initial values")
    print("R",R)
    print("M", M)
    print("v", v)
    print("d",d)
    print("---------")
    # initial values
    lam = np.array([1 for i in range(len(b))],dtype=c_double)
    mu = np.array([-np.inf for i in range(len(b))],dtype=c_double)
    lam_star =  np.array([0 for i in range(len(b))],dtype=c_double)
    print("initial lambda", lam)
    print("initial working set", W)
    print("mu",mu)
    iter = 0
    while True:
        if iter == 5:
            break
        iter += 1
        print("---------")
        print("Iteration",iter)
        W = np.sort(W)
        W = list(W)
        print("W",W, "len(W)", len(W))
        W_bar =  np.sort([x for x in range(len(lam)) if x not in W])
        print("W_bar", W_bar)
        print("mu at the beginning of the round",mu)
        p = np.array([0,0],dtype=c_double)
        if len(W)==0 or is_invertible(M[W]@ M[W].T): # step 2
            print("case: MkMkT invertible")
            print("lam_star",lam_star)
            print("lam",lam)
            print("len(W)",len(W))
            if len(W)>0:
                print("np.linalg.inv(M[W] @M[W].T)",np.linalg.inv(M[W] @M[W].T))
                print("d[W]",d[W])
                print("- np.linalg.inv(M[W] @M[W].T) @d[W]",- np.linalg.inv(M[W] @M[W].T) @d[W])
                lam_star[W] = - np.linalg.inv(M[W] @M[W].T) @d[W] # solve the system, step 3
            else:
                lam_star=np.array([0,0],dtype=c_double)
            print("lam",lam)
            print("lam_star",lam_star)
            if (np.array(lam_star) >= 0).all(): # step 4
                print("case: all lam_star>=0")
                print("len(W), len(W_bar)",len(W), len(W_bar))
                print("mu before update", mu)
                if len(W)>0:
                    if len(W_bar)>0: # constraints are split between W and W_bar
                        print("M[W_bar]",M[W_bar])
                        print("M[W]",M[W])
                        print("lam_star[W]",lam_star[W])
                        print("d[W_bar]",d[W_bar])
                        print(" M[W_bar] @M[W].T", M[W_bar] @M[W].T)
                        print("M[W_bar] @M[W].T@lam_star[W]",M[W_bar] @M[W].T@lam_star[W])
                        print("M[W_bar] @M[W].T@lam_star[W]+d[W_bar]",M[W_bar] @M[W].T@lam_star[W]+d[W_bar])
                        mu[W] = M[W_bar] @M[W].T@lam_star[W]+d[W_bar]
                    else: # all constraints in W_bar
                        print("case: all constraints in W_bar")
                        mu = np.array([0,0],dtype=c_double)
                elif len(W)==0: # all constraints in W
                    mu = np.copy(d)
                print("mu after update",mu)
                print("update lam now")
                print("mu",mu)
                lam = np.copy(lam_star) # optimum found ?
                print("lam",lam)
                print("mu",mu)
                if (np.array(mu) >= 0).all(): # step 6
                    # optimum found
                    print("case: all mu >=0")
                    print("print all optimal values",R,M,lam_star, W,iter)
                    # problem: algorithm doesn't stop, when working set contains everything
                    return -np.linalg.inv(R)@(M[W].T@lam_star[W]+v), lam_star, W, iter
                else: # step 7
                    print("case: not all mu >=0")
                    print("mu_W_bar",mu[W_bar])
                    j = np.argmin(mu[W_bar])
                    print("j",W_bar[j])
                    W.append(int(W_bar[j]))
                    print("new W for next round",W)
            else:
                print("case: lam_star<0") # step 8
                print("lam_star",lam_star)
                print("lam",lam)
                p = lam_star-lam # step 9
                print("p",p)
                B = [i for i in W if lam_star[i]<0]
                print("B",B)
                lam,lam_star,mu, W = fix_component(lam,lam_star,mu, W, B, p) # step 10
                print("lm_star",lam_star)
        else:
            print("W",W)
            print("M[W]",M[W])
            MM=Matrix(M[W] @ M[W].T) # step 12
            print("MM",MM)
            print("nullspace",MM.nullspace())
            nullspace = np.asarray(MM.nullspace()).reshape(-1)
            print("length of MM nullspace",len(nullspace))
            if len(nullspace)==0:
                p[W] = np.zeros((1,len(W)))
            else:
                for i in nullspace: #  (iterate over potential vectors in the nullspace)
                    print("element of nullspace",i)
                    pp = p
                    pp[W] = i
                    print("pp",pp.T)
                    print("d",d)
                    print("multiplication",pp.T@d)
                    if pp.T@d< 0: 
                        print("case: test constraint") 
                        p[W] = i
                        break

            print("p",p)
            
            B = [i for i in W if p[i]<0] #step 13
            lam,lam_star,mu, W =  fix_component(lam,lam_star,mu,W,B,p) # step 14
            print("lam_star",lam_star)


W = [] 


x_star, lam_star, W, iter = daqp_self(H,f,A,bupper,sense,W)  
print("final outcome", x_star, lam_star,W, "iterations", iter)
        

initial values
R [[1.         1.        ]
 [0.         1.41421356]]
M [[-2.          2.12132034]
 [-1.          2.82842712]
 [ 1.         -2.82842712]]
v [  1.         -10.60660172]
d [-17.5 -20.   35. ]
---------
initial lambda [1. 1.]
initial working set []
mu [-10. -10.]
---------
Iteration 1
W [] len(W) 0
W_bar [0 1]
mu at the beginning of the round [-10. -10.]
case: MkMkT invertible
lam_star [0. 0.]
lam [1. 1.]
len(W) 0
lam [1. 1.]
lam_star [0. 0.]
case: all lam_star>=0
len(W), len(W_bar) 0 2
mu before update [-10. -10.]
mu after update [-17.5 -20.   35. ]
update lam now
mu [-17.5 -20.   35. ]
lam [0. 0.]
mu [-17.5 -20.   35. ]
case: not all mu >=0
mu_W_bar [-17.5 -20. ]
j 1
new W for next round [1]
---------
Iteration 2
W [np.int64(1)] len(W) 1
W_bar [0]
mu at the beginning of the round [-17.5 -20.   35. ]
case: MkMkT invertible
lam_star [0. 0.]
lam [0. 0.]
len(W) 1
np.linalg.inv(M[W] @M[W].T) [[0.11111111]]
d[W] [-20.]
- np.linalg.inv(M[W] @M[W].T) @d[W] [2.22222222]
lam [0. 0.]